In [2]:
import sqlalchemy
from sqlalchemy import create_engine
engine = create_engine('sqlite:///example.db', echo = True)
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()
import pandas as pd

# Rerational datebase
## テーブル

In [3]:
from sqlalchemy import Column, Integer, String, Float, Date, DateTime, SmallInteger, Boolean
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key = True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
    
    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (self.name, self.fullname, self.password)
    
class Address(Base):
     __tablename__ = 'addresses'
     id = Column(Integer, primary_key=True)
     email_address = Column(String, nullable=False)
     user_id = Column(Integer, ForeignKey('users.id'))

     user = relationship("User", back_populates="addresses")

     def __repr__(self):
         return "<Address(email_address='%s')>" % self.email_address

User.addresses = relationship("Address", order_by=Address.id, back_populates="user")
Base.metadata.create_all(engine)

2018-01-21 11:42:55,723 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-01-21 11:42:55,729 INFO sqlalchemy.engine.base.Engine ()
2018-01-21 11:42:55,732 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-01-21 11:42:55,733 INFO sqlalchemy.engine.base.Engine ()
2018-01-21 11:42:55,771 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-01-21 11:42:55,775 INFO sqlalchemy.engine.base.Engine ()
2018-01-21 11:42:55,841 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2018-01-21 11:42:55,842 INFO sqlalchemy.engine.base.Engine ()


## データ挿入

In [4]:
jack = User(name='jack', fullname='Jack Bean', password='gjffdd')

In [5]:
bob = pd.read_csv('man.csv')

In [6]:
bob.to_sql('users', engine, index=False, if_exists='append')

2018-01-21 11:42:56,649 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-01-21 11:42:56,651 INFO sqlalchemy.engine.base.Engine ()
2018-01-21 11:42:56,656 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-01-21 11:42:56,658 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2018-01-21 11:42:56,659 INFO sqlalchemy.engine.base.Engine ('bob', 'Bob North', 'bbor')
2018-01-21 11:42:56,832 INFO sqlalchemy.engine.base.Engine COMMIT


In [7]:
jack.addresses

[]

In [8]:
jack.addresses = [
                 Address(email_address='jack@google.com'),
                 Address(email_address='j25@yahoo.com')]

In [9]:
jack.addresses[1]

<Address(email_address='j25@yahoo.com')>

In [10]:
jack.addresses[1].user

<User(name='jack', fullname='Jack Bean', password='gjffdd')>

In [11]:
session.add(jack)

In [12]:
session.commit()

2018-01-21 11:42:58,274 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-01-21 11:42:58,276 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2018-01-21 11:42:58,278 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Bean', 'gjffdd')
2018-01-21 11:42:58,373 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2018-01-21 11:42:58,374 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 4)
2018-01-21 11:42:58,376 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2018-01-21 11:42:58,378 INFO sqlalchemy.engine.base.Engine ('j25@yahoo.com', 4)
2018-01-21 11:42:58,380 INFO sqlalchemy.engine.base.Engine COMMIT


In [13]:
session.query(User).all()

2018-01-21 11:42:58,580 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-01-21 11:42:58,585 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users
2018-01-21 11:42:58,586 INFO sqlalchemy.engine.base.Engine ()


[<User(name='bob', fullname='Bob North', password='bbor')>,
 <User(name='jack', fullname='Jack Bean', password='gjffdd')>,
 <User(name='bob', fullname='Bob North', password='bbor')>,
 <User(name='jack', fullname='Jack Bean', password='gjffdd')>]

In [14]:
session.query(Address).all()

2018-01-21 11:42:58,715 INFO sqlalchemy.engine.base.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses
2018-01-21 11:42:58,717 INFO sqlalchemy.engine.base.Engine ()


[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>,
 <Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]

## データの全削除

In [15]:
#session.query(User).delete()
#session.query(Address).delete()
#session.commit()